# PyTorch Lightningインストール(PyTorch Lightning install)

In [ ]:
!pip install pytorch_lightning

     |████████████████████████████████| 923 kB 6.4 MB/s 
     |████████████████████████████████| 282 kB 64.3 MB/s 
     |████████████████████████████████| 636 kB 50.7 MB/s 
     |████████████████████████████████| 119 kB 74.1 MB/s 
     |████████████████████████████████| 829 kB 64.3 MB/s 
     |████████████████████████████████| 1.3 MB 57.3 MB/s 
     |████████████████████████████████| 142 kB 72.0 MB/s 
     |████████████████████████████████| 294 kB 67.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=ceffce65288b57af232f2ee310874c24db2555c0d97c8368204bb23d8c36656f
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

# 乱数シード固定(Random seed fixed)

In [ ]:
import os
import random

import torch
import numpy as np

def initialize_random_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
initialize_random_seed(42)

# データセットダウンロード(Download Dataset)

In [ ]:
use_sample_image = True

if use_sample_image:
    !git clone https://github.com/Kazuhito00/NanoDet-Colaboratory-Training-Sample.git

Cloning into 'NanoDet-Colaboratory-Training-Sample'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 118 (delta 48), reused 113 (delta 47), pack-reused 0
Receiving objects: 100% (118/118), 11.64 MiB | 18.71 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
import os

# 独自のデータを使用する場合は、パスを指定してください
# Please fill in the path if you want to use your own data
if use_sample_image:
    dataset_directory = 'NanoDet-Colaboratory-Training-Sample/02.annotation_data'
else:
    dataset_directory = ''

# 学習/検証データパス
train_directory = './train'
validation_directory = './validation'

# 学習データ格納ディレクトリ作成(Create training data storage directory)
for dir_path in os.listdir(dataset_directory):
    os.makedirs(train_directory + '/' + dir_path, exist_ok=True)
# 検証データ格納ディレクトリ作成(Create verification data storage directory)
for dir_path in os.listdir(dataset_directory):
    os.makedirs(validation_directory + '/' + dir_path, exist_ok=True)

In [ ]:
import glob
import shutil

# 学習データの割合(Percentage of training data)
train_ratio = 0.8

# コピー元ファイルリスト取得(Get copy source file list)
annotation_list = sorted(glob.glob(dataset_directory + '/Annotations/*'))
image_list = sorted(glob.glob(dataset_directory + '/JPEGImages/*'))

file_num = len(annotation_list)

# インデックスシャッフル(shuffle)
index_list = list(range(file_num - 1))
random.shuffle(index_list)

for count, index in enumerate(index_list):
    if count < int(file_num * train_ratio):
        # 学習用データ(Training Data)
        shutil.copy2(annotation_list[index], train_directory + '/Annotations')
        shutil.copy2(image_list[index], train_directory + '/JPEGImages')
    else:
        # 検証用データ(Validation Data)
        shutil.copy2(annotation_list[index], validation_directory + '/Annotations')
        shutil.copy2(image_list[index], validation_directory + '/JPEGImages')

# Pascal VOC形式 を MS COCO形式へ変換

In [ ]:
!git clone https://github.com/Kazuhito00/voc2coco.git

Cloning into 'voc2coco'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 53 (delta 0), reused 1 (delta 0), pack-reused 50
Unpacking objects: 100% (53/53), done.


In [ ]:
!python voc2coco/voc2coco.py train/Annotations train/annotations.json
!python voc2coco/voc2coco.py validation/Annotations validation/annotations.json

Number of xml files: 40
Success: train/annotations.json
Number of xml files: 10
Success: validation/annotations.json


# モデル訓練(Training Model)

In [ ]:
!git clone https://github.com/RangiLyu/nanodet.git
%cd nanodet
!cp tools/train.py ./

Cloning into 'nanodet'...
remote: Enumerating objects: 2246, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 2246 (delta 21), reused 37 (delta 9), pack-reused 2162
Receiving objects: 100% (2246/2246), 5.27 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (1264/1264), done.
/content/nanodet


「nanodet-m.yml」を格納してください(Store "nanodet-m.yml")<br><br>
![image](https://user-images.githubusercontent.com/37477845/133949475-e6aefaab-fbcd-41fd-a059-b99999bbe5a3.png)

In [ ]:
# 訓練(Training)
!python train.py nanodet-m.yml

[root][09-20 01:28:11]INFO:Using Tensorboard, logs will be saved in workspace/nanodet_m/logs
[root][09-20 01:28:13]INFO:Setting up data...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
[root][09-20 01:28:13]INFO:Creating model...
model size is  1.0x
init weights...
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to /root/.cache/torch/hub/checkpoints/shufflenetv2_x1-5666bf0f80.pth
100% 8.

# ONNX変換(Convert to ONNX)

In [ ]:
!cp tools/export_onnx.py ./

In [ ]:
!python export_onnx.py \
    --cfg_path=nanodet-m.yml \
    --model_path=workspace/nanodet_m/model_best/model_best.ckpt

model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth
Finish initialize NanoDet Head.
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Trig

In [ ]:
!pip install onnx-simplifier

     |████████████████████████████████| 12.3 MB 6.6 MB/s 
     |████████████████████████████████| 466 kB 70.2 MB/s 
     |████████████████████████████████| 4.5 MB 61.9 MB/s 
  Created wheel for onnx-simplifier: filename=onnx_simplifier-0.3.6-py3-none-any.whl size=12873 sha256=9bf9a2fe7d913b6b7b8e32f2bdb4897b9f4ebbfaccb964aabbf208c9a80355f7
  Stored in directory: /root/.cache/pip/wheels/0c/47/80/8eb21098e22c19d60b1c14021ee67442b4ad2d7991fdad46ba
Successfully built onnx-simplifier


In [ ]:
!python -m onnxsim nanodet.onnx nanodet.onnx

Simplifying...
Checking 0/3...
Checking 1/3...
Checking 2/3...
Ok!


# ONNXファイル情報確認(Check ONNX file information)

In [ ]:
!pip install onnxruntime

In [ ]:
import onnxruntime

onnx_session = onnxruntime.InferenceSession('nanodet.onnx')

In [ ]:
input_detail = onnx_session.get_inputs()
output_detail = onnx_session.get_outputs()

input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name

print(len(input_detail), len(output_detail))
print('input_detail: ', input_detail[0])
print('output_detail: ', output_detail[0])
print('output_detail: ', output_detail[1])
print('output_detail: ', output_detail[2])
print('output_detail: ', output_detail[3])
print('output_detail: ', output_detail[4])
print('output_detail: ', output_detail[5])

1 6
input_detail:  NodeArg(name='input.1', type='tensor(float)', shape=[1, 3, 320, 320])
output_detail:  NodeArg(name='cls_pred_stride_8', type='tensor(float)', shape=[1, 1600, 1])
output_detail:  NodeArg(name='cls_pred_stride_16', type='tensor(float)', shape=[1, 400, 1])
output_detail:  NodeArg(name='cls_pred_stride_32', type='tensor(float)', shape=[1, 100, 1])
output_detail:  NodeArg(name='dis_pred_stride_8', type='tensor(float)', shape=[1, 1600, 32])
output_detail:  NodeArg(name='dis_pred_stride_16', type='tensor(float)', shape=[1, 400, 32])
output_detail:  NodeArg(name='dis_pred_stride_32', type='tensor(float)', shape=[1, 100, 32])


# 学習済ファイルダウンロード(Download Trained Model)

In [ ]:
!cp 'nanodet.onnx' 'workspace/nanodet_m'

In [ ]:
# ダウンロードする
from google.colab import files

!zip -r 'workspace.zip' 'workspace'
files.download('workspace.zip')

  adding: workspace/ (stored 0%)
  adding: workspace/nanodet_m/ (stored 0%)
  adding: workspace/nanodet_m/logs/ (stored 0%)
  adding: workspace/nanodet_m/logs/events.out.tfevents.1632101293.81a6e6149bdf.176.0 (deflated 5%)
  adding: workspace/nanodet_m/lightning_logs/ (stored 0%)
  adding: workspace/nanodet_m/lightning_logs/version_0/ (stored 0%)
  adding: workspace/nanodet_m/lightning_logs/version_0/Train_loss_loss_dfl_Train/ (stored 0%)
  adding: workspace/nanodet_m/lightning_logs/version_0/Train_loss_loss_dfl_Train/events.out.tfevents.1632101301.81a6e6149bdf.176.5 (deflated 62%)
  adding: workspace/nanodet_m/lightning_logs/version_0/Train_loss_lr_Train/ (stored 0%)
  adding: workspace/nanodet_m/lightning_logs/version_0/Train_loss_lr_Train/events.out.tfevents.1632101301.81a6e6149bdf.176.2 (deflated 58%)
  adding: workspace/nanodet_m/lightning_logs/version_0/Val_metrics_AP_75_Val/ (stored 0%)
  adding: workspace/nanodet_m/lightning_logs/version_0/Val_metrics_AP_75_Val/events.out.tfeve

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>